In [21]:
# Importing the libraries
import numpy as np
import pandas as pd
import tensorflow as tf

# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


In [22]:
# Encoding categorical data
# Label Encoding the "Gender" column

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer
le = LabelEncoder()
X[:,1] = le.fit_transform(X[:, 1])#country
print(X)
X[:,2] = le.fit_transform(X[:, 2])#gender


[[619 0 'Female' ... 1 1 101348.88]
 [608 2 'Female' ... 0 1 112542.58]
 [502 0 'Female' ... 1 0 113931.57]
 ...
 [709 0 'Female' ... 0 1 42085.58]
 [772 1 'Male' ... 1 0 92888.52]
 [792 0 'Female' ... 1 0 38190.78]]


In [35]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')# dummies for country
X = np.array(ct.fit_transform(X))
print(X)
# dummies for country


[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 1.0 0.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [1.0 0.0 1.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [36]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [37]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


# Part 2 - Building the ANN

In [38]:
# Part 2 - Building the ANN

# Initializing the ANN
import keras
from keras.models import Sequential
from keras.layers import Dense

# Initializing the ANN

ann = Sequential()#?
num_columns = X.shape[1]

print("Number of columns:", num_columns)
# Adding the First hidden layer
ann.add(Dense(units=6, activation='relu'))# o/p dm = 6 as (11+1)/2 = 6 where input dm is 11


Number of columns: 12


In [39]:
# Adding the second hidden layer
ann.add(Dense(units=6, activation='relu'))

In [40]:
# Adding the output layer
ann.add(Dense(units=1, activation='sigmoid'))

In [41]:
# Part 3 - Training the ANN

# Compiling the ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# Training the ANN on the Training set
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.5929 - accuracy: 0.7640
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4956 - accuracy: 0.7974
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4651 - accuracy: 0.7966
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4505 - accuracy: 0.8002
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4413 - accuracy: 0.8023
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4346 - accuracy: 0.8065
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4291 - accuracy: 0.8092
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4238 - accuracy: 0.8125
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4191 - accuracy: 0.8148
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4152 - accura

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer?

In [42]:

print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 189ms/step
[[False]]


# Predicting the Test set results

In [43]:

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))



63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


# Exporting results to a CSV file

In [44]:
# Creating a DataFrame with predictions and actual labels
results_df = pd.DataFrame({'Predicted': y_pred.flatten(), 'Actual': y_test})

# Exporting results to a CSV file
results_df.to_csv('prediction_results.csv', index=False)

# # Making the Confusion Matrix

In [45]:

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1509   86]
 [ 187  218]]


0.8635

# Plotting a bar chart to visualize characteristics of people leaving the bank